# Stellar Metallicity Distribution

In [13]:
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
from scipy.stats import binned_statistic
from auriga.snapshot import Snapshot
from auriga.images import figure_setup, set_axs_configuration
from auriga.settings import Settings
from auriga.parser import parse

In [14]:
figure_setup()
settings = Settings()

In [15]:
def read_data(simulation: str, of: str, to: str) -> tuple:
    """
    This method returns data of interest for this analysis.

    Parameters
    ----------
    simulation : str
        The simulation to consider.

    Returns
    -------
    pd.DataFrame
        A data frame with the properties.
    """

    settings = Settings()

    s = Snapshot(simulation=simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
    s.tag_particles_by_region(
        disc_std_circ=settings.disc_std_circ,
        disc_min_circ=settings.disc_min_circ,
        cold_disc_delta_circ=settings.cold_disc_delta_circ,
        bulge_max_specific_energy=settings.bulge_max_specific_energy)
    s.add_metal_abundance(of, to)

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    df = pd.DataFrame()
    df["[Fe/H]"] = s.metal_abundance["Fe/H"][is_real_star & is_main_obj]
    df["RegTag"] = s.region_tag[is_real_star & is_main_obj]

    return df

## [Fe/H] Abundance

In [46]:
def add_panel_for_galaxy(simulation: str, ax: plt.Axes):
    """
    This method adds a panel with the results for a given simulation.

    Parameters
    ----------
    simulation : str
        The simulation.
    ax : plt.Axes
        The ax to which to add the plot.
    """

    galaxy, _, _, _ = parse(simulation=simulation)
    df = read_data(simulation=simulation, of="Fe", to="H")

    for tag in [2, 3]:
        is_region = (df["RegTag"] == tag)
        n_bins = 100
        ax.hist(df["[Fe/H]"][is_region], bins=n_bins, range=ax.get_xlim(),
                density=True, histtype="stepfilled", alpha=0.5, lw=0.0, 
                color=settings.component_colors[settings.components[tag]])
        ax.hist(df["[Fe/H]"][is_region], bins=n_bins, range=ax.get_xlim(),
                density=True, histtype="step", lw=1.0, 
                color=settings.component_colors[settings.components[tag]],
                label=settings.component_labels[settings.components[tag]])
    ax.text(x=0.95, y=0.95, size=6.0,
            s=r"$\texttt{" + f"au{galaxy}_or_l4".upper() + "}$",
            ha='right', va='top', transform=ax.transAxes)

In [56]:
def plot_metall_dist_for_sample(simulations: list, filename: str):
    """
    This method creates a plot of the abundance distribution for each
    simulation.

    Parameters
    ----------
    simulations : str
        A list of simulations to plot.
    filename : str
        The name of the output file.
    """

    n_simulations = len(simulations)

    fig = plt.figure(figsize=(7.2, 7.2))
    gs = fig.add_gridspec(nrows=6, ncols=5, hspace=0.0, wspace=0.0)
    axs = gs.subplots(sharex=True, sharey=True)

    set_axs_configuration(
        xlim=(-2, 2), ylim=(0, 2.5),
        xticks=[-1, 0, 1], yticks=[0, 0.5, 1, 1.5, 2],
        xlabel="[Fe/H]", ylabel="PDF",
        axs=axs, n_used=n_simulations,
        )

    for idx, ax in enumerate(axs.flat):
        # ax.grid(True, ls='--', lw=0.2, c='silver', zorder=-10)
        if idx < n_simulations:
            add_panel_for_galaxy(simulations[idx], ax=ax)
        else:
            ax.axis("off")

    axs[0, 0].legend(loc="upper left", framealpha=0.0, fontsize=4.0)

    for ax in axs.flatten():
        ax.xaxis.label.set_size(8.0)
        ax.yaxis.label.set_size(8.0)

    fig.savefig(f"../images/metallicity_distribution/{filename}.pdf")
    plt.close(fig)

In [58]:
# Create plots for all galaxies in two figures
originals = [f"au{i}_or_l4_s127" for i in settings.galaxies]
reruns = [f"au{i}_re_l4_s251" for i in settings.reruns]
plot_metall_dist_for_sample(simulations=originals, filename="originals")
plot_metall_dist_for_sample(simulations=reruns, filename="reruns")